In [ ]:
!pip install -U -q PyDrive
!pip install ijson
!pip install numpy pandas matplotlib

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Replace with your shared drive ID
shared_drive_id = '1WcjMv_jvTMpqeI5N4IKa0g6TbeR-h3aY'

# List files in the shared drive
file_list = drive.ListFile({'q': f"'{shared_drive_id}' in parents and trashed=false"}).GetList()

for file in file_list:
    print(f"Title: {file['title']}, ID: {file['id']}")

In [ ]:
# Example: Download a file from the shared drive
file_id = '1uSoQRTZq6KWbFS6aX63w3WNQaiQQf0B_'  # Replace with the file ID
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(downloaded['title'])  # Download the file
print(f"Downloaded: {downloaded['title']}")

In [ ]:
# Example: Download a file from the shared drive
file_id = '1oxCDgVNoHgWWYHC9z5eicliLr8O7-Odh'  # Replace with the file ID
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(downloaded['title'])  # Download the file
print(f"Downloaded: {downloaded['title']}")

In [ ]:
!unzip /content/amplitude_export_2024.zip -d /content/2024_json

In [ ]:
!unzip /content/amplitude_export_2025.json.zip -d /content/2025_json

Archive:  /content/amplitude_export_2025.json.zip
  inflating: /content/2025_json/new_amplitude_export_2025.json  

In [ ]:
import pandas as pd
import os
from pathlib import Path

# File year
year = 2024
folder_path = f"2024_json/new_export"

# Non-empty columns
columns_keep = [
    "client_event_time",
    "client_upload_time",
    "event_time",
    "event_type",
    "processed_time",
    "server_received_time",
    "server_upload_time",
    "session_id",
    "user_id",
]

# Create output directory
path = Path(f"{year}_csv")
if not path.exists():
    path.mkdir(parents=True, exist_ok=True)

# Process files in chunks
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    chunk_size = 100000  # Adjust based on your memory constraints
    chunk_count = 0

    # Read JSON file in chunks
    for chunk in pd.read_json(file_path, lines=True, chunksize=chunk_size):
        # Filter columns and clean data
        chunk = chunk[columns_keep]
        chunk = chunk.replace("EMPTY", pd.NA).dropna()

        # Save chunk to CSV
        output_csv = f"{year}_csv/{file_name.split('.')[0]}_chunk_{chunk_count}.csv"
        chunk.to_csv(output_csv, index=False)
        chunk_count += 1

        # Optionally, append to a master DataFrame (if needed)
        # mass2024df = pd.concat([mass2024df, chunk], ignore_index=True)

# If you need to combine all chunks into a single DataFrame, load them back in chunks
mass2024df = pd.concat(
    (pd.read_csv(f) for f in path.glob("*.csv")),
    ignore_index=True
)

# Analyze event counts
event_counts2024 = mass2024df['event_type'].value_counts()
threshold24 = 0.001 * len(mass2024df)
common_event_types2024 = event_counts2024[event_counts2024 > threshold24].index.tolist()
filtered_events2024 = event_counts2024[event_counts2024 > threshold24]

# Plot
filtered_events2024.plot(kind='bar', title='Class Distribution (Events > 0.1% of the time)')
plt.xlabel("Event Type")
plt.ylabel("Count")
plt.show()

ValueError: Expected object or value

In [ ]:
import ijson
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

#file year
year = 2025
file_path = f"2025_json/new_amplitude_export_2025.json"

#non-empty columns
columns_keep = [
    "client_event_time",
    "client_upload_time",
    "event_time",
    "event_type",
    "processed_time",
    "server_received_time",
    "server_upload_time",
    "session_id",
    "user_id",
]
mass2025df = pd.DataFrame()
path = Path(f"{year}_csv")
if not path.exists():
    path.mkdir(parents=True, exist_ok=True)
#use ijson to read the json files efficiently in memory
with open(file_path, "r") as f:
    objects = ijson.items(f, "item") #creates a generator object

    batch_size = 100000 #can be updated, currently saves per batches of 100,000
    chunk = []
    count = 0 #used to index batch file
    for obj in objects:
        chunk.append(obj)
        if len(chunk) >= batch_size:
            df = pd.DataFrame(chunk)
            output_csv = f"{year}_csv/amplitude_export_chunk_{count*batch_size}_{(count+1)*batch_size}.csv"
            df = df[columns_keep] #remove empty columns
            df = df.replace("EMPTY", pd.NA).dropna()
            mass2025df = pd.concat([mass2025df, df], ignore_index=True)
            df.to_csv(output_csv, index=False)
            count += 1
            chunk = []

    if chunk: #process remaining data if any
        output_csv = f"{year}_csv/amplitude_export_chunk_{count*batch_size}_{(count+1)*batch_size}.csv"
        df = pd.DataFrame(chunk)
        df = df[columns_keep]
        df = df.replace("None", pd.NA).dropna()
        mass2025df = pd.concat([mass2025df, df], ignore_index=True)
        df.to_csv(output_csv, index=False)
event_counts = mass2025df['event_type'].value_counts()
threshold = 0.001 * len(mass2025df)
common_event_types = event_counts[event_counts > threshold].index.tolist()
filtered_events = event_counts[event_counts > threshold]
filtered_events.plot(kind='bar', title='Class Distribution (Events > 0.1% of the time)')
plt.xlabel("Event Type")
plt.ylabel("Count")
plt.show()

This model is for the 2025 preprocessed dataframe.

In [ ]:
from collections import defaultdict

# Assuming df is your cleaned DataFrame
# Sort the data by session_id and event_time
mass2025df = mass2025df.sort_values(by=['session_id', 'event_time'])

# Create the event_before column
mass2025df['event_before'] = mass2025df.groupby('session_id')['event_type'].shift()

# Drop the first row of each session since it won't have an event_before
mass2025df = mass2025df.dropna(subset=['event_before'])

# Prepare the data for Markov chain training
transition_counts = defaultdict(lambda: defaultdict(int))

for _, row in mass2025df.iterrows():
    transition_counts[row['event_before']][row['event_type']] += 1

# Convert counts to probabilities
transition_probabilities = defaultdict(dict)

for event_before, transitions in transition_counts.items():
    total = sum(transitions.values())
    for event_type, count in transitions.items():
        transition_probabilities[event_before][event_type] = count / total

# Now you have a transition probability matrix
# You can use this to predict the next event given the current event

# Example: Predict the next event given the current event
def predict_next_event(current_event):
    if current_event in transition_probabilities:
        next_event_probs = transition_probabilities[current_event]
        return max(next_event_probs, key=next_event_probs.get)
    else:
        return None

# Example usage
event_predictions = {"current_event": [], "next_event": []}

for current_event in common_event_types:
    next_event = predict_next_event(current_event)
    event_predictions["current_event"].append(current_event)
    event_predictions["next_event"].append(next_event)

event_df = pd.DataFrame(event_predictions)

display(event_df)